In [ ]:
import os
import re
import SimpleITK as sitk

nnUNet_predict -i /mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/imagesTs -o /mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/imagesTsPred -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task600_Cartilage_Labrum

In [ ]:
imagePath = '/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/imagesTs/'
gtlabelPath = '/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/labelsTs/'
predlabelPath = '/mnt/Data/nnUNet_raw/nnUNet_raw_data/Task600_Cartilage_Labrum/imagesTsPred/'

data = []
for file in os.listdir(imagePath):
    num =int(re.search(r'\d{3}', file).group(0))
    data.append({'ID':num,
                'image':os.path.join(imagePath,file),
                 'gt':os.path.join(gtlabelPath,'labrum_{0:03d}.nii.gz'.format(num)),
                 'pred':os.path.join(predlabelPath,'labrum_{0:03d}.nii.gz'.format(num))
                })
data

In [ ]:
len(data)

In [ ]:
import medpy.metric as metric
import numpy as np
metrics = np.zeros([len(data),3])
for p, patient in enumerate(data):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(patient['gt']))
    pred = sitk.GetArrayFromImage(sitk.ReadImage(patient['pred']))
    metrics[p,0] = patient['ID']
    for i in range(1,3):
        predlabel = pred == i
        gtlabel =  gt == i
        metrics[p,i] = metric.dc(predlabel,gtlabel)

In [ ]:
print(len(metrics),np.mean(metrics[:,1:3],axis=0))
print(len(metrics),np.std(metrics[:,1:3],axis=0))
print(len(metrics),np.min(metrics[:,1:3],axis=0))
print(len(metrics),np.max(metrics[:,1:3],axis=0))


In [ ]:
metrics